In [1]:
import pickle
from tf_keras.preprocessing.text import Tokenizer

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

2025-05-31 19:59:00.813184: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-31 19:59:00.813890: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-31 19:59:00.817196: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-31 19:59:00.827853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748689140.845447   26152 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748689140.85

In [2]:
from tf_keras.models import load_model
model = load_model('rnn.keras')

2025-05-31 19:59:04.546154: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
from kiwipiepy import Kiwi
from pandas import pandas as pd
from cleanutils import clean_message

kiwi = Kiwi()

def get_tokens(text):
    if pd.isna(text) or not text:
        return []
    text = clean_message(text, False)
    tokens = kiwi.tokenize(str(text))
    filtered_tokens = []
    for token in tokens:
        if token.tag[0] in ['N', 'V', 'M', 'S', 'U']:
            filtered_tokens.append(token.form)
    return filtered_tokens

In [4]:
queries = [
    "[대한통운] 19523523912 금일 13시 배송예정. 담당기사 박* (*-*-*))",
    "무엇이 기다리고 있나요? 저는 아파트에 배송 했어요.",
    "신!규3만-지-급 출1금OK 첫 츙+ 뽀5너스 1000+",
    "안녕하세요, 고객님. 신규 회원가입을 축하드립니다. 추가 3만원 쿠폰을 받으려면?",
    "축하합니다 고객님! 문자를 받으신 분께만 깜짝 10,000원 적립금을 드렸어요.",
    "tel: vji32d 안녕하세요 고객님, 문자를 받으신 분들은 적립금을 드리니 확인해 보세요."
]

In [5]:
tokens = [get_tokens(q) for q in queries]
print(tokens)
tokens = tokenizer.texts_to_sequences(tokens)

max_len = 200
from tf_keras.preprocessing.sequence import pad_sequences
tokens = pad_sequences(tokens, maxlen=max_len)

results = model.predict(tokens)
for i, probability in enumerate(results):
    percentage = probability[0] * 100
    print(f"{percentage:.0f}%, {queries[i]}")

[['[', '대한통운', ']', '19523523912', '금일', '13', '시', '배송', '예정', '.', '담당', '기사', '박', '*', '(', '*', '-', '*', '-', '*', ')', ')'], ['무엇', '기다리', '있', '?', '저', '아파트', '배송', '.'], ['신', '!', '규3', '-', '지', '-', '급', '추', '1', '금', 'ok', '첫', '츙', '+', '뽀5', '너스', '1000', '+'], ['안녕하세요', ',', '고객', '.', '신규', '회원', '가입', '축하', '드리', '.', '추가', '3', '만', '원', '쿠폰', '받', '?'], ['축하', '고객', '!', '문자', '받', '분', '깜짝', '10,000', '원', '적립금', '드리', '.'], ['tel', ':', 'vji', '32', 'd', '안녕하세요', '고객', ',', '문자', '받', '분', '적립금', '드리', '확인', '보', '.']]
1/1 [==============================] - 0s 158ms/step
93%, [대한통운] 19523523912 금일 13시 배송예정. 담당기사 박* (*-*-*))
0%, 무엇이 기다리고 있나요? 저는 아파트에 배송 했어요.
100%, 신!규3만-지-급 출1금OK 첫 츙+ 뽀5너스 1000+
100%, 안녕하세요, 고객님. 신규 회원가입을 축하드립니다. 추가 3만원 쿠폰을 받으려면?
33%, 축하합니다 고객님! 문자를 받으신 분께만 깜짝 10,000원 적립금을 드렸어요.
99%, tel: vji32d 안녕하세요 고객님, 문자를 받으신 분들은 적립금을 드리니 확인해 보세요.
